In [15]:
import pandas as pd
import numpy as np
import ta

# Single Candlestick Patterns Functions

In [16]:
def Check_Marubozu(Shadow_u,Shadow_l,Open,Close):
    State=0
    if Shadow_u<0.1 and Shadow_l<0.1:
        if Open<Close:
            State=1
        else:
            State=-1
    return State

In [17]:
def Check_doji(Shadow_u,Shadow_l):
    if Shadow_u >4 and Shadow_l>4:
        return "Doji Yes"
    else:
        return "Doji No"

In [18]:
def Check_SCP(Open,High,Low,Close,A):
    if Open<Close:
        Shadow_u=(High-Close)/(Close-Open)
        Shadow_l=(Open-Low)/(Close-Open)
    else :
        Shadow_u=(High-Open)/(Open-Close+1)
        Shadow_l=(Close-Low)/(Open-Close+1)
    A.append(Check_Marubozu(Shadow_u,Shadow_l,Open,Close))
    
    #A.append(Check_doji(Shadow_u,Shadow_l))
    return A

# Indicator Functions

In [19]:
def Check_RSI(Open,High,Low,Close,UpperBound,LowerBound):
    
    RSI=ta.momentum.RSIIndicator(Close).rsi()
    Rsi_Check=RSI.iloc[-4:-1]
    State=0
    for i in Rsi_Check:
        if i >= UpperBound:
            State=-1
        elif i <= LowerBound:
            State=+1
    
    return State

In [20]:
def Check_MACD_Cross(Open,High,Low,Close):
    MACD=ta.trend.MACD(Close).macd()
    State=0
    if MACD.iloc[-1]>=0:
        if MACD.iloc[-2]<0:
            State= 1
    elif MACD.iloc[-1]<=0:
        if MACD.iloc[-2]>0:
            State= -1

    return State

In [21]:
def Check_MACD_Invert(Open,High,Low,Close):
    MACD=ta.trend.MACD(Close).macd()
    MACD=MACD[-7:].tolist()
    State=0
    if MACD[-1]!=max(MACD) and MACD[-1]!=min(MACD):
        if MACD[-2]-MACD[-1]>0:
            State=1
        else :
            State= -1
    return State

In [22]:
def Check_BollingerBands(Open,High,Low,Close):
    State = 0
    State = -1*max(ta.volatility.bollinger_hband_indicator(Close).iloc[-1],0)
   
    if State==0:
        State=max(ta.volatility.bollinger_lband_indicator(Close).iloc[-1],0)

    return State

In [23]:
def Check_Volume(Volume):
    VEMA=ta.trend.ema_indicator(Volume,20)
    if Volume.iloc[-1]>VEMA.iloc[-1]:
        return "Volume Yes"
    else :
        return "Volume No"

In [24]:
def Check_Indicators(Open,High,Low,Close,Volume,A):
    #Check RSI
    A.append(Check_RSI(Open,High,Low,Close,80,20))
    A.append(Check_RSI(Open,High,Low,Close,70,30))
    
    #Check MACD
    A.append(Check_MACD_Cross(Open,High,Low,Close))
    A.append(Check_MACD_Invert(Open,High,Low,Close))
    #Check Boollinger Bands
    A.append(Check_BollingerBands(Open,High,Low,Close))
    
    #A.append(Check_Volume(Volume))
    
    return A

# Backtest Function

In [ ]:
TechnicalAnalysis=pd.DataFrame([],columns=['Name','Marubozu','RSI Low Risk','RSI High Risk','MACD Cross','MACD Invert','Bollinger Bands'])
Backtest_Success={'Marubozu':0,'RSI Low Risk':0,'RSI High Risk':0,'MACD Cross':0,'MACD Invert':0,'Bollinger Bands':0}
Backtest_Total={'Marubozu':0,'RSI Low Risk':0,'RSI High Risk':0,'MACD Cross':0,'MACD Invert':0,'Bollinger Bands':0}

Watchlist_tickers=pd.read_csv('Data/Watchlist_yahoo_tickers.csv')
Watchlist_list=Watchlist_tickers["Tickers"]

takeprofit=0.03
stoploss=0.03

for name in Watchlist_list:
    print(f'Iterating Stock {name}')
    df = pd.read_excel('Data\Backtest_Data.xlsx',sheet_name= name )
    df.fillna(0)
    n=df.shape[0]-5
    for i in range(40,n):

        A=[]
        #Checking with patters
        Open=df["Open"].iloc[i]
        High=df["High"].iloc[i]
        Low=df["Low"].iloc[i]
        Close=df["Close"].iloc[i]
        Check_SCP(Open,High,Low,Close,A)
        #Checking with indicators
        Open=df["Open"].iloc[0:i]
        High=df["High"].iloc[0:i]
        Low=df["Low"].iloc[0:i]
        Close=df["Close"].iloc[0:i]
        Volume=df["Volume"].iloc[0:i]
        Check_Indicators(Open,High,Low,Close,Volume,A)
        
        #stoploss calculations
        k=0
        for j in A:
            slhit=False
            tphit=False
            if j==1:
                maxHigh=max(df['High'].iloc[i:i+4])
                minLow= min(df['Low'].iloc[i:i+4])
                Upside=(maxHigh-df["Close"].iloc[i])/df["Close"].iloc[i]
                Downside=(df["Close"].iloc[i]-minLow)/df["Close"].iloc[i]
                if Upside>=takeprofit:
                    tphit=True
                if Downside>=stoploss:
                    slhit=True
                if  tphit==True:    
                    Backtest_Success[list(Backtest_Success.keys())[k]]+=1
                Backtest_Total[list(Backtest_Success.keys())[k]]+=1
            k=k+1

print(Backtest_Success)

                      

Iterating Stock ADANIPORTS.NS
Iterating Stock ASIANPAINT.NS
Iterating Stock AXISBANK.NS
Iterating Stock BAJAJ-AUTO.NS
Iterating Stock BAJFINANCE.NS
Iterating Stock BAJAJFINSV.NS
Iterating Stock BPCL.NS
Iterating Stock BHARTIARTL.NS
Iterating Stock BRITANNIA.NS


In [ ]:
print(Backtest_Success)